# Test of the NEB Tm Calculator ([tmcalculator.neb.com](https://tmcalculator.neb.com/#!/batch)) 

    example data in text format         Tm (NEB Phusion)
    P1fwd	 AGCGGATAACAATTTCACACAGGA   62
    P1rev	 GTAAAACGACGGCCAGT          57
    pBeta	 AGCGGATAACAATTTCAC         52
    P3fwd	 AGCGGATAAGGGCAATTTCAC      61
    P3rev	 GTAAAACGACGGCCA            53

Taq DNA Polymerase with [Standard Taq Buffer](https://www.neb.com/en/products/m0273-taq-dna-polymerase-with-standard-taq-buffer).

    1X Standard Taq Reaction Buffer Pack
    10 mM Tris-HCl
    50 mM KCl
    1.5 mM MgCl2
    (pH 8.3 @ 25°C)

The final concentration of dNTPs is typically 200 μM of each deoxynucleotide [source](https://www.neb.com/en/protocols/0001/01/01/taq-dna-polymerase-with-standard-taq-buffer-m0273).

Thermodynamic data from SantaLucia 1998 and salt correction of Owczarzy 2004 were used according to docs [here](https://tmcalculator.neb.com/#!/help) and [here](https://tmapi.neb.com/#usage).

SantaLucia, J, Jr. “A Unified View of Polymer, Dumbbell, and Oligonucleotide DNA Nearest-Neighbor Thermodynamics.” Proc. Natl. Acad. Sci. U. S. A. 95, no. 4 (February 17, 1998): 1460–65. https://www.ncbi.nlm.nih.gov/pubmed/9465037.

Owczarzy, Richard, Yong You, Bernardo G Moreira, Jeffrey A Manthey, Lingyan Huang, Mark A Behlke, and Joseph A Walder. “Effects of Sodium Ions on DNA Duplex Oligomers: Improved Predictions of Melting Temperatures.” Biochemistry 43, no. 12 (March 30, 2004): 3537–54. https://doi.org/10.1021/bi034621r.

PDFs of there references can be found in the refs folder of this repository.

Can the NEB Tm Calculator be replicated in Biopython? There are four [built in tables](https://biopython.org/docs/1.83/api/Bio.SeqUtils.MeltingTemp.html#Bio.SeqUtils.MeltingTemp.Tm_NN): 

- DNA_NN1: values from Breslauer et al. (1986)
- DNA_NN2: values from Sugimoto et al. (1996)
- DNA_NN3: values from Allawi & SantaLucia (1997) (default) < == > Same as SantaLucia 1998
- DNA_NN4: values from SantaLucia & Hicks (2004)

Biopython does not have the SantaLucia (1998) thermodynamic data. This is similar, but not identical to DNA_NN3 and DNA_NN4 in the table above.

Salt correction from BioPython [docs](https://biopython.org/docs/1.83/api/Bio.SeqUtils.MeltingTemp.html#Bio.SeqUtils.MeltingTemp.salt_correction): 
    
1. 16.6 x log[Na+](Schildkraut & Lifson (1965), Biopolymers 3: 195-208)
2. 16.6 x log([Na+]/(1.0 + 0.7*[Na+]))(Wetmur (1991), Crit Rev Biochem Mol Biol 126: 227-259)
3. 12.5 x log(Na+](SantaLucia et al. (1996), Biochemistry 35: 3555-3562
4. 11.7 x log[Na+](SantaLucia (1998), Proc Natl Acad Sci USA 95: 1460-1465
5. Correction for deltaS: 0.368 x (N-1) x ln[Na+] (SantaLucia (1998), Proc Natl Acad Sci USA 95: 1460-1465)
6. (4.29(%GC)-3.95)x1e-5 x ln[Na+] + 9.40e-6 x ln[Na+]^2  (Owczarzy et al. (2004), Biochemistry 43: 3537-3554) <====<<<<<
7. Complex formula with decision tree and 7 empirical constants. Mg2+ is corrected for dNTPs binding (if present) (Owczarzy et al. (2008), Biochemistry 47: 5336-5353)

The NEB calculator probably uses the #6 Owczarzy et al. (2004).
   

In [2]:
from pydna.parsers import parse_primers

In [3]:
primers = parse_primers("""
>P1fwd NEB Tm Calculator=58
AGCGGATAACAATTTCACACAGGA	
>P1rev NEB Tm Calculator=54
GTAAAACGACGGCCAGT	       
>pBeta NEB Tm Calculator=48
AGCGGATAACAATTTCAC
>P3fwd NEB Tm Calculator=57
AGCGGATAAGGGCAATTTCAC	    
>P3rev NEB Tm Calculator=50
GTAAAACGACGGCCA	        
""")

In [4]:
from Bio.SeqUtils import MeltingTemp as mt

In [5]:
table_dict = {
"Breslauer 1986 " :mt.DNA_NN1, 
"Sugimoto 1996" :mt.DNA_NN2, 
"Allawi and SantaLucia 1997" :mt.DNA_NN3, 
"SantaLucia & Hicks 2004" :mt.DNA_NN4,
}

In [6]:
saltcorr_dict = { "Schildkraut 1965":1,
                  "Wetmur 1991":2,
                  "SantaLucia 1996":3,
                  "SantaLucia 1998":4,
                  "SantaLucia 1998 dS":5,
                  "Owczarzy 2004":6,
                  "Owczarzy 2008":7, }

In [7]:
nn_table = table_dict["Allawi and SantaLucia 1997"]
saltcorr = saltcorr_dict["Owczarzy 2004"]

for p in primers:

    tm = mt.Tm_NN(str(p.seq), 
                  nn_table=nn_table,
                  Na=50,         # mM
                  Tris=10,       # mM
                  Mg=1.5,        # mM
                  dnac1=200,     # nM
                  dnac2=0,       # nM
                  dNTPs=200*4,     # mM
                  saltcorr=saltcorr)
    
    print(p.description, f"biopython={round(tm)}   ({tm})")

P1fwd NEB Tm Calculator=58 biopython=58   (57.614086700105645)
P1rev NEB Tm Calculator=54 biopython=54   (53.96469180796623)
pBeta NEB Tm Calculator=48 biopython=48   (47.95215317414028)
P3fwd NEB Tm Calculator=57 biopython=57   (56.948297500718695)
P3rev NEB Tm Calculator=50 biopython=50   (50.361977364511176)
